<a href="https://colab.research.google.com/github/yumakemore/ApproximateName2RxNorm/blob/master/RxNorm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#ApproximateName2RxNorm

Byunggu Yu, Ph.D., April 1, 2020

Given an approximate medication concept name, this program finds the best matching concept from RxNorm and returns the details such as standard codes, standard name, dossage, etc. 

Note. This is a per-record solution. If you need to treat a set of many records, check out  https://github.com/yumakemore/Medication2RxNorm

Example:

Input:

zocor 10 mg


Output:

RxNorm ApproximateTerm Score =  75

---------------------------------------------

TTY = SBD

PRESCRIBABLE = Y

HUMAN_DRUG = US

SCHEDULE = 0

AVAILABLE_STRENGTH = 10 MG

GENERAL_CARDINALITY = SINGLE

RxCUI = 104490

NDA = NDA019766

UMLSCUI = C0354662

MMSL_CODE = BD3081

SPL_SET_ID = fdbfe194-b845-42c5-bb87-a48118bc72e7

RxNorm Name = Simvastatin 10 MG Oral Tablet [Zocor]

RxNorm Synonym = Zocor 10 MG Oral Tablet

Prescribable Synonym = Zocor 10 MG Oral Tablet

Source = Gold Standard Alchemy

Source = Multum MediSource Lexicon

Source = Micromedex RED BOOK

Source = Metathesaurus FDA Structured Product Labels



In [0]:
import requests
import json

def ApproximateName2RxNorm(approximate_name):
  #e.g., approximate_name="zocor 10 mg"
  name_in_url=approximate_name.replace(' ','%20')
  TermRequest = "https://rxnav.nlm.nih.gov/REST/approximateTerm.json?term="+name_in_url+"&maxEntries=1"

  resp = requests.get(TermRequest)
  if resp.status_code != 200:
      # This means something went wrong.
      raise ApiError('GET /tasks/ {}'.format(resp.status_code))


  #print(resp.text)
  items = resp.json()
  rxcui=[]
  rxcui_approximateTerm_score=[]
  rxcui_attributes_text=[]
  rxcui_attributes_json=[]
  for item in items['approximateGroup']['candidate']:
    #print(item)
    #print(item['rxcui'])
    #print("RxNorm ApproximateTerm Score = ",item['score'])
    rxcui.append(item['rxcui'])
    rxcui_request = "https://rxnav.nlm.nih.gov/REST/rxcui/"+item['rxcui']+"/allProperties.json?prop=all"
    rxcui_resp = requests.get(rxcui_request)
    if rxcui_resp.status_code != 200:
      # This means something went wrong.
      raise ApiError('GET /tasks/ {}'.format(rxcui_resp.status_code))

    #print(rxcui_resp.text, "\n")
    rxcui_attributes_text.append(rxcui_resp.text)
    rxcui_attributes_json.append(rxcui_resp.json())
    rxcui_approximateTerm_score.append(item['score'])

  selected_concept=-1
  for i in range(0,len(rxcui_attributes_json)):
    #print("All Attributes of RxCUI=",rxcui[i],": ",rxcui_attributes_text[i])
    for j in range(0, len(rxcui_attributes_json[i]['propConceptGroup']['propConcept'])):
      #print(rxcui_attributes_json[i]['propConceptGroup']['propConcept'][j]['propName'],"=",
      #      rxcui_attributes_json[i]['propConceptGroup']['propConcept'][j]['propValue'])
      if rxcui_attributes_json[i]['propConceptGroup']['propConcept'][j]['propName']=="NDA":
        selected_concept=i
        break
    if selected_concept != -1: break

  return rxcui_attributes_json[selected_concept], rxcui_approximateTerm_score[selected_concept]
    

In [0]:
Myname = input("Please enter a concept name (e.g., zocor 10 mg):\n")
print("\n... Wait. I am querying RxNorm ...\n")
RxNormData, score =ApproximateName2RxNorm(Myname)

print("RxNorm ApproximateTerm Score = ",score)
print("---------------------------------------------")
for j in range(0, len(RxNormData['propConceptGroup']['propConcept'])):
  print(RxNormData['propConceptGroup']['propConcept'][j]['propName'],"=",
        RxNormData['propConceptGroup']['propConcept'][j]['propValue'])


Please enter a concept name (e.g., zocor 10 mg):
tylen ol

... Wait. I am querying RxNorm ...

RxNorm ApproximateTerm Score =  50
---------------------------------------------
TTY = BN
PRESCRIBABLE = Y
BN_CARDINALITY = single
GENERAL_CARDINALITY = SINGLE
RxCUI = 202433
UMLSCUI = C0699142
MESH = D000082
MMSL_CODE = BN6732
RxNorm Name = Tylenol
Source = Multum MediSource Lexicon
Source = Medical Subject Headings
